In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import process_time
nov2015 = pd.read_csv(r"C:/Users/losaa/Downloads/Escritorio/Estudios/PROYECTO III/PAR-2015-Nov1-Nov28.csv", sep = ",")

C:\Users\losaa\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
class Summary:
    def __init__(self, data, *args):
        self._summary = data
        
    def get_descr(self, colname):
        
        """Returns the description of a specific variable"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Description"].values[0]
    
    def get_type(self, colname):
        
        """Returns the type of a specific variable"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Type"].values[0]
    
    def get_ifna(self, colname):
        
        """Returns whether a specific variable allows NA values"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Accepts NAs"].values[0]
    
    def get_cols_by_type(self, type):
        
        """Returns the names of the variables of a certain type. If you want the output
        to be a list, you must transform it to list (list(get_cols_by_type('int')))"""
        
        return self._summary.loc[self._summary["Type"] == type, "Variable"]

In [3]:
summary = Summary(pd.read_csv(r"C:/Users/losaa/Downloads/Escritorio/Estudios/PROYECTO III/VariablesDescr.csv", sep = ";", nrows = 15))

In [4]:
for var in list(summary.get_cols_by_type("datetime")):
    nov2015[var] = pd.to_datetime(nov2015[var])


#Categories
for var in list(summary.get_cols_by_type("factor")):
    nov2015[var] = nov2015[var].astype("category")

nov2015.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5193201 entries, 0 to 5193200
Data columns (total 15 columns):
 #   Column           Dtype         
---  ------           -----         
 0   bookingdate      datetime64[ns]
 1   arrivaldate      datetime64[ns]
 2   poocountry       category      
 3   origincity       category      
 4   destinationcity  category      
 5   bookingsign      category      
 6   leadtime         int64         
 7   paxprofile       category      
 8   lengthofstay     int64         
 9   losname          category      
 10  cabinclass       category      
 11  distchannel      category      
 12  pax              int64         
 13  numpss           int64         
 14  numnss           int64         
dtypes: category(8), datetime64[ns](2), int64(5)
memory usage: 326.9 MB


In [5]:
data = {}

for cat in np.unique(nov2015.paxprofile):
    data[cat] = nov2015.iloc[:,7:8].applymap(lambda x: x == cat).values

In [6]:
for columna in data:
    nov2015[columna] = data[columna]

In [7]:
nov2015.head()

,bookingdate,arrivaldate,poocountry,origincity,destinationcity,bookingsign,leadtime,paxprofile,lengthofstay,losname,cabinclass,distchannel,pax,numpss,numnss,BUSINESS,GROUP,LEISURE,VFR
0,2015-11-08,2016-01-05,AR,SAO,PAR,PARTIAL_CANCELLATION,57,LEISURE,-2,DWELLING_TRANSFER,T,OTHER,-1,1,4,False,False,True,False
1,2015-11-08,2015-11-18,FR,NYC,PAR,NEW_BOOKING,9,BUSINESS,-3,RETURN_HOME,T,OTHER,1,1,0,True,False,False,False
2,2015-11-08,2016-06-03,FR,YYC,PAR,PARTIAL_CANCELLATION,207,LEISURE,-3,RETURN_HOME,T,OTHER,-3,2,0,False,False,True,False
3,2015-11-08,2015-11-09,US,MIA,PAR,PARTIAL_ADDITION,0,LEISURE,-4,SHORT_TRANSFER,T,OTHER,1,1,4,False,False,True,False
4,2015-11-08,2015-11-23,DZ,ALG,PAR,NEW_BOOKING,15,LEISURE,13,STAY,B,OTHER,3,0,1,False,False,True,False


In [8]:
nov2015 = nov2015.loc[((nov2015['bookingdate']) >= '2015-11-13') & ((nov2015['bookingdate']) <= '2015-11-23')]

In [9]:
numnov2015 = nov2015.loc[:, list(summary.get_cols_by_type("int"))]
boolean = nov2015.iloc[:,15:]

In [10]:
for columna in boolean.columns.values:
    numnov2015[columna] = boolean[columna]

In [11]:
numnov2015['bookingsign'] = nov2015['bookingsign']

In [12]:
numnov2015.head()

,leadtime,lengthofstay,pax,numnss,numpss,BUSINESS,GROUP,LEISURE,VFR,bookingsign
409,35,-2,-1,1,2,False,False,True,False,PARTIAL_CANCELLATION
410,112,-1,1,0,0,False,False,True,False,NEW_BOOKING
411,105,-4,-1,1,2,False,False,True,False,FULL_CANCELLATION
412,12,-3,1,0,0,False,False,True,False,NEW_BOOKING
413,23,-4,2,5,0,False,False,True,False,PARTIAL_ADDITION


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(numnov2015.iloc[:,:-1],numnov2015.bookingsign, test_size = 0.2, random_state = 88)

In [21]:
Y_test.to_csv('Y_test.csv', header=False, index=False)

In [14]:
X_train = X_train[1:1000]
Y_train = Y_train[1:1000]

In [15]:
from sklearn.model_selection import cross_val_score
import math
import pandas as pd
from sklearn import svm

def plot_accuracy(nombres, nn_list_scores,errorInterval):
    pd.DataFrame({"Tipo":nombres,"Error":nn_list_scores}).set_index("Tipo").plot.bar(yerr=errorInterval,figsize= (9,6),ylim=(0.0,0.5),rot=0)
    plt.xticks(rotation=90)
    plt.ylim(0,0.15)
    plt.show()

cv_scores=[]
nombres=[]


clf = svm.SVC(kernel='linear')
clf.fit(X_train,Y_train)

#for i in [ 'rbf', 'poly', 'linear']:
    #for c in [100]:
        
        #if i=='poly':
            
            #for d in [2]:
                #print(f' Entro con grado:   {d}')
                #clf = svm.SVC(kernel=i, C=c, degree=d)
                #clf.fit(X_train,Y_train)
                
                
    
        
                
#errorInterval = [1.96*math.sqrt(x*(1-x)/len(Y)) for x in cv_scores]
#print(errorInterval)
#plot_accuracy(nombres, cv_scores,errorInterval)

C:\Users\losaa\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\losaa\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [16]:
prediction = clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, prediction)

0.726056432740673